In [1]:
import folium
from datosgobmx import client
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt

In [69]:
################################################################################
# Module: Station visualizer
# developed by: Edgar Egurrola
# 			  edgar.egurrola@tec.mx
# updated: 28/05/2020
################################################################################

import folium
from datosgobmx import client
import pandas as pd
import matplotlib.pyplot as plt

def pollutant(p):
    """Function that returns a str with a pollutant.

    Args:
        p (int): values from 0 to 5 for list place.

    Returns:
        str: pollutant.
    """
    #Parametros de contaminantes
    param = ['CO','NO2', 'O3','PM10','SO2']
    return (param[p])

def compare_aq(p, city):

    dir_pcs = '../data/processed/'
    dir_pcs_cat = dir_pcs+'aqip_'+city #Directory to save concatenation
    dir_fig_cmp = '../output/figures/aqip_analysis/'

    compare = pd.read_csv(dir_pcs_cat +city+'_AQIP.csv')
    compare = compare.drop(columns=['count','min','max','median','variance','PARAM','FECHA']).rename(columns={'Specie':'Contaminante',
                                                                                                                'Date':'Fecha'})
    ax = plt.gca()
    compare[compare['Contaminante']==p].plot(kind='scatter', x='Fecha',y='cdmx_median', color='green', ax=ax)
    compare[compare['Contaminante']==p].plot(kind='scatter', x='Fecha',y='aqip_median', color='orange', alpha = 0.75, ax=ax)
    
    plt.ylabel('Concentration: '+p)
    
    plt.savefig(dir_fig_cmp+p+'_x1.png')
    
    #plt.show()

def visualize_stations():
    
    parametros_request = client.makeCall('sinaica-estaciones',{'pageSize':200})

    estaciones = []

    for v in parametros_request['results']:
        aux = pd.DataFrame.from_dict(v,orient='index').T
        estaciones.append(aux)

    estaciones = pd.concat(estaciones, ignore_index=True)
    
    #Quita las estaciones que esten fuera de Mexico
    mask = (estaciones.lat.between(14, 34.5)) & (estaciones.long.between(-120, -70))
    estaciones = estaciones[mask]

    #La variable estaciones son los datos obtenidos de la API, en formato Pandas Dataframe

    centro_lat, centro_lon = 22.396092, -101.731430 #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=5,
                            tiles = 'cartodb positron')

    #Se colocan los puntos de las estaciones
    for i, row in estaciones.iterrows():

        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {row.nombre} <br> <b> Latitud: </b> {row.lat:.5f} <br> <b> Longitud: </b> {row.long:.5f} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[row.lat, row.long], radius=5,
                            tooltip = popup_text, fill=True, fill_opacity=0.4).add_to(folium_map)

    return folium_map


def graph_yearly(city):
    """Creates a graph that compares the air quality data for each pollutant from 2017 to 2020

    Args:
        city (str): code for the city to be analysed, for example: cdmx
    """
    dir_pcs = '../data/processed/'
    dir_fig = '../output/figures/'


    years = [2017, 2018, 2019, 2020] #Years to be referenced

    year_dict = {2017:'green', 2018:'blue',
                2019:'orange', 2020:'red'}
    
    data_csv = dir_pcs+city+'/'+city+'_2017-2019_'
    
    if city == 'cdmx':
        data_csv = dir_pcs+city+'/'+city+'_2017-2020_filtered_'
    
    
    for i in range(5):
        
        data_mean = pd.read_csv(data_csv+pollutant(i)+'.csv').set_index('FECHA')
        
        data_mean['mean'] = data_mean.mean(axis=1)
        
        data_mean = data_mean.reset_index()
        
        data_mean['FECHA'] = pd.to_datetime(data_mean['FECHA'])
    
        
        ax = plt.gca()
        
        for y in years:
            filter_year=data_mean[data_mean['FECHA'].dt.year==y]
            
            filter_year['FECHA'] = filter_year['FECHA'].dt.strftime('%m-%d')
            
            #print(filter_year)
            
            filter_year.plot(x='FECHA',y='mean', color=year_dict[y], label = str(y), ax=ax)
            
            
        plt.ylabel('Concentration: '+pollutant(i))
        
        plt.savefig(dir_fig+city+'_yearly/Year_Compare_'+pollutant(i)+'.png')
        
        ax.clear()

def colors(conc_nrm):
    """Receives a normalized concentration value and returns a hex depending on the air quality level.

    Args:
        conc_nrm (float): result of the division between the averaged concentration and a bad quality concentration.

    Returns:
        str : hex color
    """

    if conc_nrm<=0.25:
        color = '#3CB371'
    elif conc_nrm<=0.5:
        color = '#FFD700'
    elif conc_nrm<=0.75:
        color = '#FF7F50'
    else:
        color = '#483D8B'
        
    return color

def imeca_colors (param, conc):
    
    imeca = 0
    
    if param == 'CO':
    
        imeca = conc * 100 / 11
        
    elif param == 'SO2':
    
        imeca = (conc/1000) * 100 / 0.11
        
    elif param == 'NO2':
    
        imeca = (conc/1000) * 100 / 0.21
        
    elif param == 'O3':
        
        conc = conc/1000
        
        if conc <= 0.07:
            imeca = 714.29*conc
        
        elif 0.07 < conc <= 0.095:
            imeca = 2041.67*(conc-0.071)+51
            
        elif 0.095 < conc <= 0.154:
            imeca = 844.83*(conc-0.096)+101
            
        elif 0.154 < conc <= 0.204:
            imeca = 1000*(conc-0.155)+151
            
        elif 0.204 < conc <= 0.404:
            imeca = 497.49*(conc-0.205)+201
        
        elif 0.404 < conc:
            imeca = 1000*(conc-104)
            
    elif param == 'PM10':
        
        if conc <= 40:
            imeca = 1.25*conc
        
        elif 40 < conc <= 75:
            imeca = 1.44*(conc-41)+51
            
        elif 75 < conc <= 214:
            imeca = 0.355*(conc-76)+101
            
        elif 214 < conc <= 354:
            imeca = 0.353*(conc-215)+151
            
        elif 354 < conc <= 424:
            imeca = 1.4359*(conc-355)+201
        
        elif 424 < conc <= 504:
            imeca = 1.253*(conc-425)+301
            
        elif 504 < conc:
            imeca = conc-104
            
    if imeca <= 50:
        color = '#75b46f'
        
    elif 50 < imeca <= 100:
        color = '#f7ff55'
        
    elif 100 < imeca <= 150:
        color = '#ff9e4f'
        
    elif 150  < imeca <= 200:
        color = '#db3331'
    
    elif 200 < imeca:
        color = '#c158b8'
        
    else:
        color = '#ffffff'
        
    return (color)
                
    

def visualize_aqdata_date(city, pollutant, date):
    """ Creates a folium map with stations arranged by size and colour depending on the concentration of the pollutant.

    Args:
        city (str): code for the city to be analysed, for example: cdmx
        pollutant (str): pollutant to be plotted
        date (str): date to be analysed

    Returns:
        folium map
    """
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México',
                'gdl':'Guadalajara'}
    
    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}
    
    data_csv = dir_pcs+city+'/'+city+'_2017-2019_'+pollutant+'.csv'
    
    if city == 'cdmx':
        data_csv = dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv'
    
    data_bydateParam = pd.read_csv(data_csv).set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)
    
    cnt_x = (min_x+max_x)/2
    cnt_y = (min_y+max_y)/2
    
    
    centro_lat, centro_lon = cnt_y, cnt_x #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=10,
                            tiles = 'cartodb positron')

    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

        est_code = city_stations.loc[(i),'codigo']

        #Coloca los marcadores en el mapa
        c_value = data_bydateParam.loc[(date),est_code]
        c_graph = (data_bydateParam.loc[(date),est_code])/p_limits[pollutant]

        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {est.nombre} <br> <b> Latitud: </b> {est.lat:.5f} <br> <b> Longitud: </b> {est.long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {c_value} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[est.lat, est.long], radius=c_graph*50,
                            tooltip = popup_text, fill=True, color=imeca_colors(pollutant, c_value),
                            fill_opacity=0.65).add_to(folium_map)

    return(folium_map)

def clr_change(conc_cmp):
    """Function that receives a float of a normalyzed concentration and returns a hex.

    Args:
        conc_cmp (float): value resulted from the division bewteen the concentration of the year analysed and the year prior.

    Returns:
        str: hex with the code, if the value of the current year is smaller it's blue, otherwise it's red.
    """
    
    if conc_cmp>=0:
        color = '#ff7d55'
    elif conc_cmp<0:
        color = '#55ff6c'
    else:
        color = '#ffffff'
        
    return color

def compare_year_prior(city, pollutant, date):
    """Creates a map with the analysis of the average between the input date and the previous year for a selected pollutant.

    Args:
        city (str): code for the city to be analysed, for example: cdmx
        pollutant (str): pollutant to be plotted
        date (str): date to be analysed

    Returns:
        folium map where a blue marker indicates a smaller value of the input date concentration and, the bigger the marker the larger the concentration
    """
    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    dir_pcs = '../data/processed/'

    city_dict = {'cdmx':'Valle de México',
                'gdl':'Guadalajara'}
    
    prev_year = str(int(date[:4])-1)+date[4:]
    
    data_csv = dir_pcs+city+'/'+city+'_2017-2019_'+pollutant+'.csv'
    
    if city == 'cdmx':
        data_csv = dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv'
    
    data_bydateParam = pd.read_csv(data_csv).set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)
    
    cnt_x = (min_x+max_x)/2
    cnt_y = (min_y+max_y)/2
    
    
    centro_lat, centro_lon = cnt_y, cnt_x #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=10,
                            tiles = 'cartodb positron')

    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

        est_code = city_stations.loc[(i),'codigo']

        #Coloca los marcadores en el mapa
        c_current = data_bydateParam.loc[(date),est_code]
        c_prev = data_bydateParam.loc[(prev_year),est_code]
        c_graph = (c_current - c_prev)/c_prev

        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {est.nombre} <br> <b> Latitud: </b> {est.lat:.5f} <br> <b> Longitud: </b> {est.long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Cambio: </b> {c_graph} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[est.lat, est.long], radius=abs(c_graph)*50,
                            tooltip = popup_text, fill=True, color=clr_change(c_graph),
                            fill_opacity=0.65).add_to(folium_map)

    return(folium_map)

def interpolate_aqdata_date(city, pollutant, date):
    """Function that creates a folium map with user specified city, pollutant and date and interpolates valid values.


        Args:
        city (str): city code to be analyzed (i.e. cdmx).
        pollutant (str): pollutant to be analyzed (i.e. PM10).
        date (str): date to be analyzed with format yyyy-mm-dd.
    """
    dir_pcs = '../data/processed/'

    city_stations = pd.read_csv('../data/raw/Grl/stations/city_stations.csv')

    city_dict = {'cdmx':'Valle de México',
                'gdl':'Guadalajara'}

    p_limits = {'PM10':214, 'O3':154, 'CO':16.5,
               'PM25':97.4, 'SO2':195,'NO2':315}

    data_csv = dir_pcs+city+'/'+city+'_2017-2019_'+pollutant+'.csv'
    
    if city == 'cdmx':
        data_csv = dir_pcs+city+'/'+city+'_2017-2020_filtered_'+pollutant+'.csv'
    
    data_bydateParam = pd.read_csv(data_csv).set_index('FECHA')
    
    #lists to append valid values of lat and long for interpolation
    
    x = []
    y = []
    
    for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]
                
                if pd.notna(c_value):
                    x.append(est.long)
                    y.append(est.lat)

    #Registers the boundries coordinates for the interpolation
    min_x = min(x)
    max_x = max(x)
    min_y = min(y)
    max_y = max(y)

    folium_map = visualize_aqdata_date(city, pollutant, date)

    #Valor de celda
    cellsize = 0.01
    #Valor de potencia
    p = 2

    #x and y values for the start of the interpolation
    xidw=min_x
    yidw=min_y

    #Variables for interpolation
    dividendo = 0
    divisor = 0
    idw=[]

    while xidw <= max_x:
        while yidw <= max_y:
            for i, est in city_stations[city_stations['city']==city_dict[city]].iterrows():

                est_code = city_stations.loc[(i),'codigo']
                c_value = data_bydateParam.loc[(date),est_code]

                if pd.notna(c_value):
                    dividendo = (c_value/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p)))+ dividendo
                    divisor = (1/(sqrt((est.long-xidw)**2+(est.lat-yidw)**2)**(p))) + divisor

            concentracion = dividendo/divisor
            #Aqui se guardan los valores de las concentraciones para usarlos despues

            c_graph = concentracion/p_limits[pollutant]

            #Puntos con nombre, latitud y longitud
            popup_text = f"<b> Nombre: </b> {'NA'} <br> <b> Latitud: </b> {yidw:.5f} <br> <b> Longitud: </b> {xidw:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {concentracion} <br>"

            #Coloca los marcadores en el mapa
            folium.CircleMarker(location=[yidw, xidw], radius=1,
                                tooltip = popup_text, fill=True, color=imeca_colors(pollutant, concentracion),
                                opacity=0.45).add_to(folium_map)

            idw.append([yidw,xidw,concentracion])
            yidw = yidw + cellsize
            dividendo = 0
            divisor = 0  
        xidw = xidw + cellsize
        yidw = min_y

    inter = pd.DataFrame(idw, columns=['lat','long','conc'])


    return(folium_map)


def visualize_mx_aqip(pollutant, date):

    
    
    dir_ext = '../data/external/INEGI/'
    
    dir_pcs = '../data/processed/'
    
    mx_mun = pd.read_csv(dir_ext+'CentroidMunicipalities_INEGI19_v1.csv', encoding ='latin_1')
    
    aqip_mx = pd.read_csv(dir_pcs+'aqip/MX_'+pollutant+'_2017-2020.csv', encoding='latin_1').set_index('Date')
    
    
    centro_lat, centro_lon = 22.396092, -101.731430 #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=5,
                            tiles = 'cartodb positron')
    
    cty_aqip = [c for c in aqip_mx.columns]
    
    for i in range(1,len(cty_aqip)):
        
        city = cty_aqip[i]
        
        if city == 'Oaxaca':
            city = 'Oaxaca de Juárez'
        
        elif city == 'Mexico City':
            city = 'Ciudad de México'
        
        c_value = aqip_mx.loc[date,cty_aqip[i]]
        
        lat = float(mx_mun[mx_mun['NOMGEO']==city].lat)
        #lat = lat[mx_mun[mx_mun['NOMGEO']==cty_aqip[i]].index.values[0]]
        
        long = float(mx_mun[mx_mun['NOMGEO']==city].long)
        #long = long[mx_mun[mx_mun['NOMGEO']==cty_aqip[i]].index.values[0]]
 
        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {city} <br> <b> Latitud: </b> {lat:.5f} <br> <b> Longitud: </b> {long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Conc: </b> {c_value} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[lat, long], radius=5,
                            tooltip = popup_text, fill=True, color=imeca_colors(pollutant, c_value),
                            fill_opacity=0.65).add_to(folium_map)

    return(folium_map)


def compare_mx_aqip(pollutant, date):

    
    
    dir_ext = '../data/external/INEGI/'
    
    dir_pcs = '../data/processed/'
    
    mx_mun = pd.read_csv(dir_ext+'CentroidMunicipalities_INEGI19_v1.csv', encoding ='latin_1')
    
    aqip_mx = pd.read_csv(dir_pcs+'aqip/MX_'+pollutant+'_2017-2020.csv', encoding='latin_1').set_index('Date')
    
    prev_year = str(int(date[:4])-1)+date[4:]
    
    centro_lat, centro_lon = 22.396092, -101.731430 #Centro del mapa

    #Creacion del mapa
    folium_map = folium.Map(location=[centro_lat,centro_lon], zoom_start=5,
                            tiles = 'cartodb positron')
    
    cty_aqip = [c for c in aqip_mx.columns]
    
    for i in range(1,len(cty_aqip)):
        
        city = cty_aqip[i]
        
        if city == 'Oaxaca':
            city = 'Oaxaca de Juárez'
        
        elif city == 'Mexico City':
            city = 'Ciudad de México'
        
        c_current = aqip_mx.loc[date,cty_aqip[i]]
        c_prev = aqip_mx.loc[prev_year,cty_aqip[i]]
        c_graph = (c_current - c_prev)/c_prev
        
        lat = float(mx_mun[mx_mun['NOMGEO']==city].lat)
        #lat = lat[mx_mun[mx_mun['NOMGEO']==cty_aqip[i]].index.values[0]]
        
        long = float(mx_mun[mx_mun['NOMGEO']==city].long)
        #long = long[mx_mun[mx_mun['NOMGEO']==cty_aqip[i]].index.values[0]]
 
        #Puntos con nombre, latitud y longitud
        popup_text = f"<b> Nombre: </b> {city} <br> <b> Latitud: </b> {lat:.5f} <br> <b> Longitud: </b> {long:.5f} <br> <b> Contaminante: </b> {pollutant} <br> <b> Cambio: </b> {c_graph} <br>"

        #Coloca los marcadores en el mapa
        folium.CircleMarker(location=[lat, long], radius=5,
                            tooltip = popup_text, fill=True, color=clr_change(c_graph),
                            fill_opacity=0.65).add_to(folium_map)

    return(folium_map)

In [53]:
dir_out = '../output/html/'

In [57]:
cty,param,date = ('gdl', 'O3', '2019-04-30')
folium_map = visualize_aqdata_date(cty, param, date)
folium_map.save(dir_out + cty+'_'+param+'_'+date+'.html')
folium_map

In [66]:
cty,param,date = ('gdl', 'PM10', '2018-01-10')
folium_map = compare_year_prior(cty, param, date)
folium_map.save(dir_out +'Compare_'+cty+'_'+param+'_'+date+'.html')
folium_map

In [73]:
cty,param,date = ('gdl', 'PM10', '2017-04-15')
folium_map = interpolate_aqdata_date(cty, param, date)
folium_map.save(dir_out + 'Interpola_' +cty+'_'+param+'_'+date+'.html')
folium_map

In [79]:
param,date = ('PM10', '2020-03-30')
folium_map = visualize_mx_aqip(param, date)
folium_map.save(dir_out + 'MX_' +param+'_'+date+'.html')
folium_map

In [85]:
param,date=('PM10', '2020-04-15')
folium_map = compare_mx_aqip(param, date)
folium_map.save(dir_out + 'Compare_MX_' +param+'_'+date+'.html')
folium_map